In [1]:
import pandas
import json
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import time
from collections import OrderedDict
from pandas import DataFrame
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains


records = []
links_reportagem = []
titulos = []
data_hora = []
links_noticia = []
list_data_comentarios = []

session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)


driver = webdriver.Chrome('C:/chromedriver/chromedriver.exe')

def limpaArrays(array):
    igual = set()
    result = []
    for item in array:
        if item not in igual:
            igual.add(item)
            result.append(item)
    return result

def gera_json():
    
    dados_comentarios = {
        'comentario': records,
        'data_comentario': list_data_comentarios
    }
        
    Data = {
        'link_noticia': links_noticia,
        'titulo': titulos,
        'data_hora': data_hora,
        'comentarios': [dados_comentarios]
    }
    
    json_dict = {}
    data = []
    
    json_dict = OrderedDict([
            ('link_noticia', links_noticia),
            ('titulo', titulos),
            ('data_hora', data_hora),
            ('comentarios', [OrderedDict([
                                       ('comentarios', records),
                                       ('data_comentario', list_data_comentarios)])])
           ])
        

    df = DataFrame(json_dict)
    with open('pacoteteste_novo_json.json', 'w', encoding='utf-8') as file:
        df.to_json(file, force_ascii=False, orient='index')
    driver.quit()

def pega_comentarios():
    res = driver.execute_script('return document.documentElement.outerHTML')
    soup = BeautifulSoup(res, 'html.parser') 
    link_noiticia = soup.find("link",{"itemprop":"mainEntityOfPage"})
    href_link_noticia = link_noiticia.get('href')
    busca_titulo_noticia = soup.find("h1",{"class":"content-head__title"})
    titulo_noticia = busca_titulo_noticia.text
    busca_data_hora = soup.find("time",{"itemprop": "datePublished"})
    data_hora_text = busca_data_hora.text
    lista_recente = soup.find("div","glbComentarios-lista glbComentarios-lista-recentes")
    lista = lista_recente.find("ul", class_ = "glbComentarios-lista-todos")
    if(lista):
        li = lista.find_all("li", {"itemtype":"http://schema.org/UserComments"})
        for coments_ in li:
            coments = coments_.find("p", class_ = "glbComentarios-texto-comentario")
            if(coments):
                comentarios = coments.contents[0]
                comentarios.strip()
                records.append((comentarios)) 
            data_coments = coments_.find("abbr", class_ = "data cadastro")
            if(data_coments):
                data_comentarios = data_coments.get('title')
                list_data_comentarios.append((data_comentarios))
        
        data_hora.append((data_hora_text))
        titulos.append((titulo_noticia))
        links_noticia.append((href_link_noticia))
    gera_json()
                               
def botao_respostas():
    while True:
        try:
            mais_comentarios = driver.find_element_by_xpath("//*[@class='glbComentarios-lista glbComentarios-lista-recentes']/ul/li/div[1]/div/div[3]/button[not(contains(@style,'display: none'))]")
            time.sleep(3)
            if(mais_comentarios.is_displayed()):
                ActionChains(driver).move_to_element(mais_comentarios).click(mais_comentarios).perform()
        except NoSuchElementException as e:
            break
    pega_comentarios()   

def botao_carrega():
    while True:
        try:
            carrega_mais = driver.find_element_by_xpath('//*[@id="boxComentarios"]/div[4]/button')
            time.sleep(2)
            if(carrega_mais.is_displayed()):
                ActionChains(driver).move_to_element(carrega_mais).click(carrega_mais).perform()
            else:
                break
        except NoSuchElementException as e:
            break
    botao_respostas()
     

def busca_reportagens(url):
    driver.get(url)
    driver.maximize_window()
    try:
        box_comentarios = driver.find_element_by_id('boxComentarios')
        driver.execute_script('arguments[0].scrollIntoView(true);', box_comentarios)
    finally:
        try:
            element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "glbComentarios"))
            )
        finally:
            botao_carrega()
    

def __init__(url):
    busca_reportagens(url)
    
__init__('https://g1.globo.com/pop-arte/musica/rock-in-rio/2019/noticia/2019/09/29/witzel-diz-que-ha-genocidio-no-rj-e-que-vai-a-onu-pedir-punicoes-a-paises-vizinhos.ghtml')
        
          